In [23]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np

In [2]:
# ANOVA 하기 위한 코드
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# 오류 무시
import warnings
warnings.filterwarnings('ignore')

from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [3]:
# Prophet
from fbprophet import Prophet

# 그룹화 시계열
import hts   # gts가 여기에 포함됨
import matplotlib.pyplot as plt
import seaborn as sns
# import warnings
# warnings.simplefilter("ignore")

# settings
plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16,8)

In [4]:
%matplotlib inline

from pathlib import Path
import pandas as pd

from sklearn import preprocessing 
from sklearn.metrics import pairwise
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans #k-평균 모델 구축
import matplotlib.pylab as plt
import seaborn as sns
from pandas.plotting import parallel_coordinates

import statsmodels.formula.api as sm
from statsmodels.tsa import tsatools, stattools
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics import tsaplots


# 페이스북이 만든, Prophet 시계열 예측 라이브러리 이용

> ARIMA 같은 시계열 모델은 시간에 종속적인 구조를 가지지만, Prophet은 종속적이지 않고 Curve Fitting으로 문제 해결
>
> 구성요소로는 Growth, Seasonality, Holidays가 있으며 \[y(t)=g(t)+s(t)+h(t)+error\]구조를 취함
>
> 데이터를 Prophet에 맞도록 가공하기 위해 ds, y 2개의 colum이 필요함
>
> Prophet 객체는 1번만 Fit할 수 있음. default growth = 'linear'이고, 상한과 하한을 설정할 경우에는 Prophet의 growh = 'logistic'을 추가 
>
> 미래 dataframe 생성
> 
> 예측 후 forecast 결과 확인 

## month_long_term_frgn 장기체류

1. sum_resd_pop_cnt  ~ nationality group별로

2. sum_work_pop_cnt  ~ nationality group별로

3. sum_visit_pop_cnt ~ nationality group별로

In [33]:
long_df = pd.read_csv(r'C:\Users\이현아\Desktop\월별데이터 정리\month_long_term_frgn.csv')
long_df

,m,nationality,emd_cd,emd_nm,count_num,sum_resd_pop_cnt,sum_work_pop_cnt,sum_visit_pop_cnt
0,2018-01,AUS,50110590,건입동,33,0.0000,0.0000,193.1805
1,2018-01,AUS,50110256,구좌읍,40,0.0000,0.0000,251.2195
2,2018-01,AUS,50130253,남원읍,107,0.0000,0.0000,648.0345
3,2018-01,AUS,50110660,노형동,761,11855.9924,738.9594,2612.7042
4,2018-01,AUS,50130590,대륜동,563,2701.0357,1576.0168,200.4630
...,...,...,...,...,...,...,...,...
52233,2021-06,VNM,50130320,표선면,919,2646.5554,0.0000,7603.5847
52234,2021-06,VNM,50110310,한경면,1366,45349.6490,1121.2861,75816.8162
52235,2021-06,VNM,50110250,한림읍,1440,97147.6156,3820.6787,61844.2041
52236,2021-06,VNM,50110600,화북동,1375,13436.5104,2574.8044,8554.9721


In [34]:
long_df['nationality'].unique()

array(['AUS', 'BGD', 'CAN', 'CHN', 'DEU', 'EGY', 'ETC', 'FRA', 'GBR',
       'IDN', 'IND', 'JPN', 'KAZ', 'KGZ', 'KHM', 'LKA', 'MGL', 'MMR',
       'MYS', 'NGR', 'NPL', 'NZL', 'PAK', 'PHL', 'RUS', 'THA', 'TWN',
       'UKR', 'USA', 'UZB', 'VNM'], dtype=object)

### 1. grouping

In [35]:
group1 = ['CHN']
group2 = ['NPL','USA']
group3 = ['IDN']
group4 = ['VNM']
group5 = ['AUS','BGD','CAN','DEU','EGY','ETC','FRA','GBR','IND','JPN','KAZ','KGZ','KHM','LKA','MGL','MMR','MYS','NGR','NZL','PAK','PHL','RUS','THA','TWN','UKR','UZB']

In [36]:
#group1부터 group5부터 한번에 if문 돌리면 너어무 오래걸려서 좀 나눴어요 코드 ~~

In [37]:
for i in long_df.index:
    if long_df.loc[i,'nationality'] in group1:
        long_df.loc[i,'nationality_group'] = 'group1'

In [38]:
for i in long_df.index:
    if long_df.loc[i,'nationality'] in group2:
        long_df.loc[i,'nationality_group'] = 'group2'

In [39]:
for i in long_df.index:
    if long_df.loc[i,'nationality'] in group3:
        long_df.loc[i,'nationality_group'] = 'group3'

In [40]:
for i in long_df.index:
    if long_df.loc[i,'nationality'] in group4:
        long_df.loc[i,'nationality_group'] = 'group4'

In [41]:
for i in long_df.index:
    if long_df.loc[i,'nationality'] in group5:
        long_df.loc[i,'nationality_group'] = 'group5'

In [42]:
long_df

,m,nationality,emd_cd,emd_nm,count_num,sum_resd_pop_cnt,sum_work_pop_cnt,sum_visit_pop_cnt,nationality_group
0,2018-01,AUS,50110590,건입동,33,0.0000,0.0000,193.1805,group5
1,2018-01,AUS,50110256,구좌읍,40,0.0000,0.0000,251.2195,group5
2,2018-01,AUS,50130253,남원읍,107,0.0000,0.0000,648.0345,group5
3,2018-01,AUS,50110660,노형동,761,11855.9924,738.9594,2612.7042,group5
4,2018-01,AUS,50130590,대륜동,563,2701.0357,1576.0168,200.4630,group5
...,...,...,...,...,...,...,...,...,...
52233,2021-06,VNM,50130320,표선면,919,2646.5554,0.0000,7603.5847,group4
52234,2021-06,VNM,50110310,한경면,1366,45349.6490,1121.2861,75816.8162,group4
52235,2021-06,VNM,50110250,한림읍,1440,97147.6156,3820.6787,61844.2041,group4
52236,2021-06,VNM,50110600,화북동,1375,13436.5104,2574.8044,8554.9721,group4


In [43]:
long_df['nationality_group'].unique()

array(['group5', 'group1', 'group3', 'group2', 'group4'], dtype=object)

In [44]:
long_df['nationality_group'].value_counts()

group5    43208
group2     3612
group4     1806
group3     1806
group1     1806
Name: nationality_group, dtype: int64

### 데이터 전처리

In [45]:
# 우선, 시계열 데이터이므로 날짜를 datetime형식으로 변환
long_df['m'] = pd.to_datetime(long_df.m, format = '%Y-%m')
# long_df.info()

In [52]:
dong = long_df['emd_nm'].unique()

In [60]:
# 1. sum_resd_pop_cnt
# 국적(nationality) 그룹별로 예측하기
for c in dong:
    sum_july1 = 0
    sum_august1 = 0
    new_df = long_df[long_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_resd_pop_cnt' : 'y'})
    df_grouped = new_df.groupby('nationality_group')   
    df_final1 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final1 = pd.merge(df_final1, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final1 = df_final1[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + "그룹별 예측 결과")
    for i in range(5):
        sum_july1 += df_final1.iloc[-2,i]
        sum_august1 += df_final1.iloc[-1,i]
    print(f'7월의 sum_resd_pop_cnt(거주 인구 합) : {sum_july1}')
    print(f'8월의 sum_resd_pop_cnt(거주 인구 합) : {sum_august1}')

건입동그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 12821.035943812818
8월의 sum_resd_pop_cnt(거주 인구 합) : 27909.234904564364
구좌읍그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 1389.0433255996022
8월의 sum_resd_pop_cnt(거주 인구 합) : 17828.91271049444
남원읍그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 13096.222161924434
8월의 sum_resd_pop_cnt(거주 인구 합) : 5806.054177996325
노형동그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 48424.0807630004
8월의 sum_resd_pop_cnt(거주 인구 합) : 102314.82633043952
대륜동그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 1184.4923452113917
8월의 sum_resd_pop_cnt(거주 인구 합) : 12009.211175175016
대정읍그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 29245.963288561325
8월의 sum_resd_pop_cnt(거주 인구 합) : 48392.77126729214
대천동그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 3234.3282244502498
8월의 sum_resd_pop_cnt(거주 인구 합) : 2168.3921443799863
도두동그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : 2540.2066510970585
8월의 sum_resd_pop_cnt(거주 인구 합) : 7681.548230906168
동홍동그룹별 예측 결과
7월의 sum_resd_pop_cnt(거주 인구 합) : -7585.5538149914355
8월의 sum_resd_pop_cnt(

In [62]:
# 2. sum_work_pop_cnt
# 국적(nationality) 그룹별로 예측하기
for c in dong:
    sum_july2 = 0
    sum_august2 = 0
    new_df = long_df[long_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_work_pop_cnt' : 'y'})
    df_grouped = new_df.groupby('nationality_group')   
    df_final2 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final2 = pd.merge(df_final2, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final2 = df_final2[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    for i in range(5):
        sum_july2 += df_final2.iloc[-2,i]
        sum_august2 += df_final2.iloc[-1,i]
    print(f'7월의 sum_work_pop_cnt(근무 인구 합) : {sum_july2}')
    print(f'8월의 sum_work_pop_cnt(근무 인구 합) : {sum_august2}')

건입동 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 12821.035943812818
8월의 sum_work_pop_cnt(근무 인구 합) : 27909.234904564364
구좌읍 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 1389.0433255996022
8월의 sum_work_pop_cnt(근무 인구 합) : 17828.91271049444
남원읍 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 13096.222161924434
8월의 sum_work_pop_cnt(근무 인구 합) : 5806.054177996325
노형동 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 48424.0807630004
8월의 sum_work_pop_cnt(근무 인구 합) : 102314.82633043952
대륜동 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 1184.4923452113917
8월의 sum_work_pop_cnt(근무 인구 합) : 12009.211175175016
대정읍 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 29245.963288561325
8월의 sum_work_pop_cnt(근무 인구 합) : 48392.77126729214
대천동 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 3234.3282244502498
8월의 sum_work_pop_cnt(근무 인구 합) : 2168.3921443799863
도두동 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : 2540.2066510970585
8월의 sum_work_pop_cnt(근무 인구 합) : 7681.548230906168
동홍동 그룹별 예측 결과
7월의 sum_work_pop_cnt(근무 인구 합) : -7585.5538149914355
8월의 sum_work

데이터 값이 0에 근접해 있다면, 계절변동에 의해 negative(음수)의 예측값을 내놓을 수 있음. 
따라서 이경우, 음수인 예측값들은 0으로 생각.

In [64]:
# 3. sum_visit_pop_cnt
# 국적(nationality) 그룹별로 예측하기
for c in dong:
    sum_july3 = 0
    sum_august3 = 0
    new_df = long_df[long_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_visit_pop_cnt' : 'y'})
    df_grouped = new_df.groupby('nationality_group')   
    df_final3 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final3 = pd.merge(df_final3, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final3 = df_final3[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    for i in range(5):
        sum_july3 += df_final3.iloc[-2,i]
        sum_august3 += df_final3.iloc[-1,i]
    print(f'7월의 sum_visit_pop_cnt(방문 인구 합) : {sum_july3}')
    print(f'8월의 sum_visit_pop_cnt(방문 인구 합) : {sum_august3}')

건입동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 137177.78662951305
8월의 sum_visit_pop_cnt(방문 인구 합) : 218305.90620717462
구좌읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 70876.4132718484
8월의 sum_visit_pop_cnt(방문 인구 합) : 255916.61013910826
남원읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 185832.2778973061
8월의 sum_visit_pop_cnt(방문 인구 합) : 206713.06265903436
노형동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 298791.5422655778
8월의 sum_visit_pop_cnt(방문 인구 합) : 529539.9289740208
대륜동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 51683.25248822802
8월의 sum_visit_pop_cnt(방문 인구 합) : 130731.08137481188
대정읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 174989.990414961
8월의 sum_visit_pop_cnt(방문 인구 합) : 351519.63782250154
대천동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 5090.571509145679
8월의 sum_visit_pop_cnt(방문 인구 합) : 114973.79781532047
도두동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 48325.44569502522
8월의 sum_visit_pop_cnt(방문 인구 합) : 58436.4447231182
동홍동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 75879.18338654244
8월의

## month_short_term_frgn 단기체류

4. visit_pop_cnt ~ nationality group별로

In [65]:
short_df = pd.read_csv(r'C:\Users\이현아\Desktop\월별데이터 정리\month_short_term_frgn.csv')
short_df

,m,nationality,count_num,emd_cd,emd_nm,visit_pop_cnt
0,2018-01,CHN,744,50110590,건입동,30589.4398
1,2018-01,CHN,744,50110256,구좌읍,26691.1513
2,2018-01,CHN,744,50130253,남원읍,179846.0273
3,2018-01,CHN,744,50110660,노형동,145583.1902
4,2018-01,CHN,742,50130590,대륜동,22743.8459
...,...,...,...,...,...,...
16250,2021-06,VNM,126,50110320,추자면,36.6789
16251,2021-06,VNM,39,50130320,표선면,9.2223
16252,2021-06,VNM,45,50110310,한경면,11.1088
16253,2021-06,VNM,37,50110250,한림읍,7.7552


In [66]:
short_df['nationality'].unique()

array(['CHN', 'ETC', 'HKG', 'IDN', 'JPN', 'MYS', 'SGP', 'THA', 'USA',
       'VNM'], dtype=object)

### 1. grouping

In [67]:
group1= ['VNM']
group2 = ['JPN','SGP','IDN','MYS']
group3 = ['USA','CHN','ETC']
group4 = ['HKG','THA']

In [68]:
for i in short_df.index:
    if short_df.loc[i,'nationality'] in group1:
        short_df.loc[i,'nationality_group'] = 'group1'
    elif short_df.loc[i,'nationality'] in group2:
        short_df.loc[i,'nationality_group'] = 'group2'
    elif short_df.loc[i,'nationality'] in group3:
        short_df.loc[i,'nationality_group'] = 'group3'
    elif short_df.loc[i,'nationality'] in group4:
        short_df.loc[i,'nationality_group'] = 'group4'

In [69]:
short_df['nationality_group'].value_counts()

group2    6130
group3    5397
group4    3320
group1    1408
Name: nationality_group, dtype: int64

### 데이터 전처리

In [70]:
# 우선, 시계열 데이터이므로 날짜를 datetime형식으로 변환
short_df['m'] = pd.to_datetime(short_df.m, format = '%Y-%m')
# short_df.info()

In [71]:
dong = short_df['emd_nm'].unique()

In [73]:
# 1. visit_pop_cnt
# 국적(nationality) 그룹별로 예측하기
for c in dong:
    sum_july4 = 0
    sum_august4 = 0
    new_df = short_df[short_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'visit_pop_cnt' : 'y'})
    df_grouped = new_df.groupby('nationality_group')   
    df_final4 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final4 = pd.merge(df_final4, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final4 = df_final4[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    for i in range(4):
        sum_july4 += df_final4.iloc[-2,i]
        sum_august4 += df_final4.iloc[-1,i]
    print(f'7월의 sum_visit_pop_cnt(방문 인구 합) : {sum_july4}')
    print(f'8월의 sum_visit_pop_cnt(방문 인구 합) : {sum_august4}')

건입동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -151.24916704595944
8월의 sum_visit_pop_cnt(방문 인구 합) : 8442.972635677412
구좌읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -379.6980274262264
8월의 sum_visit_pop_cnt(방문 인구 합) : 1993.4241840598715
남원읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -2357.9986013616644
8월의 sum_visit_pop_cnt(방문 인구 합) : 14767.391124696576
노형동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 16971.663803826297
8월의 sum_visit_pop_cnt(방문 인구 합) : 23234.39183211902


INFO:fbprophet:n_changepoints greater than number of observations. Using 24.


대륜동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -738.3442699360312
8월의 sum_visit_pop_cnt(방문 인구 합) : 2393.603592110829


INFO:fbprophet:n_changepoints greater than number of observations. Using 18.


대정읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -10259.715522722327
8월의 sum_visit_pop_cnt(방문 인구 합) : -2837.1507411911625


...

중앙동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : nan
8월의 sum_visit_pop_cnt(방문 인구 합) : nan
천지동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : nan
8월의 sum_visit_pop_cnt(방문 인구 합) : nan
추자면 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 9384.915323086334
8월의 sum_visit_pop_cnt(방문 인구 합) : 13880.29733500572
표선면 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -1980.3453504895351
8월의 sum_visit_pop_cnt(방문 인구 합) : 2931.4202675773486
한경면 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 2407.9318149003143
8월의 sum_visit_pop_cnt(방문 인구 합) : 1988.9125614760273


INFO:fbprophet:n_changepoints greater than number of observations. Using 15.


한림읍 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : 12535.093689157999
8월의 sum_visit_pop_cnt(방문 인구 합) : 18300.143577236668


INFO:fbprophet:n_changepoints greater than number of observations. Using 10.


화북동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : nan
8월의 sum_visit_pop_cnt(방문 인구 합) : nan
효돈동 그룹별 예측 결과
7월의 sum_visit_pop_cnt(방문 인구 합) : -3981.4774126144957
8월의 sum_visit_pop_cnt(방문 인구 합) : -197.25310574473542


데이터 값이 0에 근접해 있다면, 계절변동에 의해 negative(음수)의 예측값을 내놓을 수 있음. 
따라서 이경우, 음수인 예측값들은 0으로 생각.

## month_resident_pop (거주 인구 데이터)

5. sum_resid_pop_cnt  ~ sex별로
6. sum_foreign_pop_cnt  ~ sex별로

In [5]:
# sex 그루핑 이미 나눠져 있음 !
resi_df = pd.read_csv(r'C:\Users\이현아\Desktop\월별데이터 정리\month_resident_pop.csv')
resi_df

,base_year,base_month,sex,emd_cd,emd_nm,count_num,sum_resid_pop_cnt,sum_foreign_pop_cnt,sum_total_pop_cnt
0,2018,1,남성,50110590,건입동,1,5085.0,146.0,5231.0
1,2018,1,남성,50110256,구좌읍,1,7965.0,368.0,8333.0
2,2018,1,남성,50130253,남원읍,1,9806.0,428.0,10234.0
3,2018,1,남성,50110660,노형동,1,26286.0,821.0,27107.0
4,2018,1,남성,50130590,대륜동,1,6836.0,83.0,6919.0
...,...,...,...,...,...,...,...,...,...
3607,2021,6,여성,50130320,표선면,1,6033.0,123.0,6156.0
3608,2021,6,여성,50110310,한경면,1,4531.0,100.0,4631.0
3609,2021,6,여성,50110250,한림읍,1,10341.0,1140.0,11481.0
3610,2021,6,여성,50110600,화북동,1,12238.0,161.0,12399.0


### 데이터 전처리

In [6]:
# 우선, 시계열 데이터이므로 날짜를 datetime형식으로 변환

for i in resi_df.index:
    if resi_df.loc[i,'base_month'] < 10:
        resi_df.loc[i,'base_month'] = '0'+ str(resi_df.loc[i,'base_month'])

In [7]:
for i in resi_df.index:
    resi_df.loc[i,'m'] = pd.to_datetime(str(resi_df.loc[i,'base_year'])+ str(resi_df.loc[i,'base_month']), format = '%Y%m')

In [8]:
dong = resi_df['emd_nm'].unique()

In [12]:
resim_df = resi_df[resi_df['sex'] == "남성"]    # 남성의 데이터
resif_df = resi_df[resi_df['sex'] == "여성"]    # 여성의 데이터

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1806 entries, 0 to 3568
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   base_year            1806 non-null   int64         
 1   base_month           1806 non-null   object        
 2   sex                  1806 non-null   object        
 3   emd_cd               1806 non-null   int64         
 4   emd_nm               1806 non-null   object        
 5   count_num            1806 non-null   int64         
 6   sum_resid_pop_cnt    1806 non-null   float64       
 7   sum_foreign_pop_cnt  1806 non-null   float64       
 8   sum_total_pop_cnt    1806 non-null   float64       
 9   m                    1806 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(3), object(3)
memory usage: 155.2+ KB


sum_resid_pop_cnt

In [ ]:
# 남성의 데이터
list_male_resid = [] # 총 값들 저장
for c in dong:
    sum_july5 = 0
    sum_august5 = 0
    new_df = resim_df[resim_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_resid_pop_cnt' : 'y'})
    df_final5 = pd.DataFrame()
    m = Prophet()
    m.daily_seasonality=True
    m.weekly_seasonality=True
    m.fit(new_df)
    future = m.make_future_dataframe(freq ='M', periods=2)
    forecast = m.predict(future)
    forecast = forecast.rename(columns={'yhat': 'yhat_'})
    df_final5 = pd.merge(df_final5, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)
    df_final5 = df_final5[['yhat_']]
    print(c + " 그룹별 예측 결과")
    sum_july5 += df_final5.iloc[-2,0]
    sum_august5 += df_final5.iloc[-1,0]
    print(f'7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : {sum_july5}')
    print(f'8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : {sum_august5}')
    list_male_resid.append(sum_july5)
    list_male_resid.append(sum_august5)

건입동 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 4491.325778987443
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 4455.635359692566
구좌읍 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 7884.765036708094
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 8025.119155791348
남원읍 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 9618.181694555326
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 9655.375706266965
노형동 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 26965.614204495672
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 27140.98107225937
대륜동 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 7643.582473942153
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 7660.724513561026
대정읍 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 10721.101222139992
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 10727.43200687831
대천동 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 7143.1673092147485
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 7112.3981159513805
도두동 그룹별 예측 결과
7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 1742.5585401978678
8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : 1739.

In [ ]:
# 여성의 데이터
list_female_resid = []
for c in dong:
    sum_july6 = 0
    sum_august6 = 0
    new_df = resif_df[resif_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_resid_pop_cnt' : 'y'})
    df_final6 = pd.DataFrame()
    m = Prophet()
    m.daily_seasonality=True
    m.weekly_seasonality=True
    m.fit(new_df)
    future = m.make_future_dataframe(freq ='M', periods=2)
    forecast = m.predict(future)
    forecast = forecast.rename(columns={'yhat': 'yhat_'})
    df_final6 = pd.merge(df_final6, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)
    df_final6 = df_final6[['yhat_']]
    print(c + " 그룹별 예측 결과")
    sum_july6 += df_final6.iloc[-2,0]
    sum_august6 += df_final6.iloc[-1,0]
    print(f'7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : {sum_july6}')
    print(f'8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : {sum_august6}')
    list_female_resid.append(sum_july6)
    list_female_resid.append(sum_august6)

In [ ]:
total_july_resid = []
total_august_resid = []
for i in range(86):
    if i%2 ==0:
        total_july_resid[i] = list_male_resid[i] + list_female_resid[i]
    else:
        total_august_resid[i] = list_male_resid[i] + list_female_resid[i]

j = 0
for c in dong:
    print(c + " 7월 예측값 : " + total_july_resid[j])
    print(c + " 8월 예측값 : " + total_august_resid[j])
    j+=1

sum_foreign_pop_cnt

In [ ]:
# 남성의 데이터
for c in dong:
    sum_july7 = 0
    sum_august7 = 0
    new_df = resim_df[resim_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_foreign_pop_cnt' : 'y'})
    df_final7 = pd.DataFrame()
    m = Prophet()
    m.daily_seasonality=True
    m.weekly_seasonality=True
    m.fit(new_df)
    future = m.make_future_dataframe(freq ='M', periods=2)
    forecast = m.predict(future)
    forecast = forecast.rename(columns={'yhat': 'yhat_'})
    df_final7 = pd.merge(df_final7, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)
    df_final7 = df_final7[['yhat_']]
    print(c + " 그룹별 예측 결과")
    sum_july7 += df_final7.iloc[-2,0]
    sum_august7 += df_final7.iloc[-1,0]
    print(f'7월의 sum_resid_pop_cnt(외국인 거주 인구 합) : {sum_july7}')
    print(f'8월의 sum_resid_pop_cnt(외국인 거주 인구 합) : {sum_august7}')

In [ ]:
# 여성의 데이터
for c in dong:
    sum_july8 = 0
    sum_august8 = 0
    new_df = resif_df[resif_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_foreign_pop_cnt' : 'y'})
    df_final8 = pd.DataFrame()
    m = Prophet()
    m.daily_seasonality=True
    m.weekly_seasonality=True
    m.fit(new_df)
    future = m.make_future_dataframe(freq ='M', periods=2)
    forecast = m.predict(future)
    forecast = forecast.rename(columns={'yhat': 'yhat_'})
    df_final8 = pd.merge(df_final8, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)
    df_final8 = df_final8[['yhat_']]
    print(c + " 그룹별 예측 결과")
    sum_july8 += df_final8.iloc[-2,0]
    sum_august8 += df_final8.iloc[-1,0]
    print(f'7월의 sum_resid_pop_cnt(외국인 거주 인구 합) : {sum_july8}')
    print(f'8월의 sum_resid_pop_cnt(외국인 거주 인구 합) : {sum_august8}')

In [ ]:
# 남성, 여성을 sex별로 나눈건데 중간에 오류떠서 위의 코드로 대체

In [214]:
# 1. sum_resid_pop_cnt
# sex별로 예측하기
for c in dong:
    sum_july5 = 0
    sum_august5 = 0
    new_df = resi_df[resi_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_resid_pop_cnt' : 'y'})
    df_grouped = new_df.groupby('sex')  
    df_final5 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final5 = pd.merge(df_final5, forecast.set_index('ds'), how='inner', left_index=True, right_index=True)

    df_final5 = df_final5[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    print(df_final5)
    for i in range(2):
        sum_july5 += df_final5.iloc[-2,i]
        sum_august5 += df_final5.iloc[-1,i]
    print(f'7월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : {sum_july5}')
    print(f'8월의 sum_resid_pop_cnt(주민등록 거주 인구 합) : {sum_august5}')

건입동 그룹별 예측 결과
Empty DataFrame
Columns: [yhat_남성, yhat_여성]
Index: []


ValueError: Dataframe has less than 2 non-NaN rows.

In [ ]:
# 2. sum_foreign_pop_cnt
# sex별로 예측하기
for c in dong:
    sum_july6 = 0
    sum_august6 = 0
    new_df = resi_df[resi_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_foreign_pop_cnt' : 'y'})
    df_grouped = new_df.groupby('sex')   
    df_final6 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final6 = pd.merge(df_final6, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final6 = df_final6[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    for i in range(2):
        sum_july6 += df_final6.iloc[-2,i]
        sum_august6 += df_final6.iloc[-1,i]
    print(f'7월의 sum_foreign_pop_cnt(외국인 거주 인구 합) : {sum_july6}')
    print(f'8월의 sum_foreign_pop_cnt(외국인 거주 인구 합) : {sum_august6}')

## month_card_spending (음식 관련 카드소비 데이터)

7. sum_use_cnt  ~ mct_cat_nm(업종)별로
8. sum_use_amt  ~ mct_cat_nm(업종)별로

In [ ]:
card_df = pd.read_csv(r'C:\Users\이현아\Desktop\월별데이터 정리\month_card_spending.csv',encoding='cp949')
card_df

### 1. mct_cat_nm grouping (업종 그룹화)

In [ ]:
card_df['mct_cat_nm'].unique()

In [ ]:
group1 = ['마트/슈퍼마켓','한식']
group2 = ['식품']
group3 = ['농축수산물']
group4 = ['아시아음식','간식','배달','패스트푸드','양식','부페','주점및주류판매']

In [ ]:
card_df['mct_cat_nm_group'].unique()

In [ ]:
card_df['mct_cat_nm_group'].value_counts()

### 데이터 전처리

In [ ]:
# 우선, 시계열 데이터이므로 날짜를 datetime형식으로 변환
card_df['m'] = pd.to_datetime(card_df.m, format = '%Y-%m')
# card_df.info()

In [ ]:
dong = card_df['emd_nm'].unique()

In [ ]:
# 1. sum_use_cnt
# mct_cat_nm 그룹별 (mct_cat_nm_group)로 예측하기
for c in dong:
    sum_july7 = 0
    sum_august7 = 0
    new_df = card_df[card_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_use_cnt' : 'y'})
    df_grouped = new_df.groupby('mct_cat_nm_group')   
    df_final7 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final7 = pd.merge(df_final7, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final7 = df_final7[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    for i in range(4):
        sum_july7 += df_final7.iloc[-2,i]
        sum_august7 += df_final7.iloc[-1,i]
    print(f'7월의 sum_use_cnt(결제 건수 합) : {sum_july7}')
    print(f'8월의 sum_use_cnt(결제 건수 합) : {sum_august7}')

In [ ]:
# 2. sum_use_amt
# mct_cat_nm 그룹별 (mct_cat_nm_group)로 예측하기
for c in dong:
    sum_july8 = 0
    sum_august8 = 0
    new_df = card_df[card_df['emd_nm']==c]
    new_df = new_df.rename(columns = {'m' : 'ds', 'sum_use_amt' : 'y'})
    df_grouped = new_df.groupby('mct_cat_nm_group')   
    df_final8 = pd.DataFrame()
    for g in df_grouped.groups:
        group = df_grouped.get_group(g)
        m = Prophet()
        m.daily_seasonality=True
        m.weekly_seasonality=True
        m.fit(group)
        future = m.make_future_dataframe(freq ='M', periods=2)
        forecast = m.predict(future)
        forecast = forecast.rename(columns={'yhat': 'yhat_'+g})
        df_final8 = pd.merge(df_final8, forecast.set_index('ds'), how='outer', left_index=True, right_index=True)

    df_final8 = df_final8[['yhat_' + g for g in df_grouped.groups.keys()]]
    print(c + " 그룹별 예측 결과")
    for i in range(4):
        sum_july8 += df_final8.iloc[-2,i]
        sum_august8 += df_final8.iloc[-1,i]
    print(f'7월의 sum_use_amt(결제 금액 합) : {sum_july8}')
    print(f'8월의 sum_use_amt(결제 금액 합) : {sum_august8}')

In [ ]:
# 문제점 -> Using 21처럼 나오는 것
# Well basically if you have very few rows, it will reduce the number of changepoints to make sure that there are always less 
# changepoints than data. It isn't necessarily an issue, but more to let you know that a specified setting (number of changepoints) 
# was overridden. That said, the model will be more likely to overfit the changepoints when there is very little data, so if you are 
# fitting very small time series and are seeing this warning, you may want to check that by plotting the forecasts.

dong_list = ['geonip','gujwa','namwon','nohyeong','daeryun','daejeong','daecheon','dodu','donghong','bonggae','samdo1','samdo2',
             'samyang','seohong','seongsan','songsan','ara','andeok','aewol','yeon','yeongcheon','yerae','ora','oedo','yongdam1',
             'yongdam2','udo','ido1','ido2','iho','Ildo1','Ildo2','jeongbang','jocheon','jungmun','jungang','cheonji','chuja','pyoseon',
             'hangyeong','hallim','hwabuk','hyodon']


## 계층적 시계열

3. 계층적 시계열에 넣어서 그 동의 7,8월 예측값

>  <1> sum_resd_pop_cnt
>
> <2> sum_work_pop_cnt
>
>  <3> sum_visit_pop_cnt

 #### <1> sum_resd_pop_cnt 예측값 구하기 


1. nationality_group에 따른 sum_resd_pop_cnt를 구하기

In [ ]:


# 건입동
geonip_df = geonip_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
geonip_df

In [ ]:
# 구좌읍
gujwa_df = gujwa_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 남원읍
namwon_df= namwon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 노형동
nohyeong_df = nohyeong_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 대륜동
daeryun_df = daeryun_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 대정읍
daejeong_df = daejeong_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 대천동
daecheon_df = daecheon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 도두동
dodu_df = dodu_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 동홍동
donghong_df = donghong_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 봉개동
bonggae_df = bonggae_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 삼도1동
samdo1_df = samdo1_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 삼도2동
samdo2_df = samdo2_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 삼양동
samyang_df = samyang_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 서홍동
seohong_df = seohong_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 성산읍
seongsan_df = seongsan_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 송산동
songsan_df = songsan_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 아라동
ara_df = ara_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 안덕면
andeok_df = andeok_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 애월읍
aewol_df = aewol_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 연동
yeon_df = yeon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 영천동
yeongcheon_df = yeongcheon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 예래동
yerae_df = yerae_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 오라동
ora_df = ora_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 외도동
oedo_df = oedo_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 용담1동
yongdam1_df = yongdam1_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 용담2동
yongdam2_df = yongdam2_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 우도면
udo_df = udo_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 이도1동
ido1_df = ido1_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 이도2동
ido2_df = ido2_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 이호동
iho_df = iho_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 일도1동
Ildo1_df = Ildo1_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 일도2동
Ildo2_df = Ildo2_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 정방동
jeongbang_df = jeongbang_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 조천읍
jocheon_df = jocheon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 중문동
jungmun_df = jungmun_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 중앙동
jungang_df = jungang_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 천지동
cheonji_df = cheonji_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 추자면
chuja_df = chuja_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 표선면
pyoseon_df = pyoseon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 한경면
hangyeong_df = hangyeong_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 한림읍
hallim_df = hallim_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 화북동 
hwabuk_df = hwabuk_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)
# 효돈동 
hyodon_df = hyodon_df.groupby(["m","nationality_group"])["sum_resd_pop_cnt"].sum().reset_index(drop = False)

In [ ]:
# +) 건입동 세트의 영역 살펴보기
geonip_df.groupby("nationality_group")["sum_resd_pop_cnt"].apply(set).to_frame()

2. scikit-hts 모델을 만들기 위한 dataframe 만들기

In [ ]:
# 건입동

# 가장 하위 레벨(수준) 만들기
geonip_df_bottom_level = geonip_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")

# total 레벨(수준) 만들기
geonip_df_total = geonip_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})

# df 합치기
geonip_hierarchy_df = geonip_df_bottom_level.join(geonip_df_total)
geonip_hierarchy_df.index = pd.to_datetime(geonip_hierarchy_df.index) # resample위해 인덱스 시간데이터여야

geonip_hierarchy_df = geonip_hierarchy_df.resample("MS").sum() # 달초를 알고 싶기에 start를 의미하는 S 붙임

# dataframe을 준비한 후 인덱스를 사용하여 결합하고 계층 구조의 하위 수준에서 고유한 계열 수 인쇄
print(f"하위 수준의 시계열 수: {geonip_df_bottom_level.shape[1]}")

In [ ]:
# 구좌읍
gujwa_df_bottom_level = gujwa_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
gujwa_df_total = gujwa_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
gujwa_hierarchy_df = gujwa_df_bottom_level.join(gujwa_df_total)
gujwa_hierarchy_df.index = pd.to_datetime(gujwa_hierarchy_df.index)
gujwa_hierarchy_df = gujwa_hierarchy_df.resample("MS").sum()
# 남원읍
namwon_df_bottom_level = namwon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
namwon_df_total = namwon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
namwon_hierarchy_df = namwon_df_bottom_level.join(namwon_df_total)
namwon_hierarchy_df.index = pd.to_datetime(namwon_hierarchy_df.index)
namwon_hierarchy_df = namwon_hierarchy_df.resample("MS").sum()
# 노형동
nohyeong_df_bottom_level = nohyeong_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
nohyeong_df_total = nohyeong_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
nohyeong_hierarchy_df = nohyeong_df_bottom_level.join(nohyeong_df_total)
nohyeong_hierarchy_df.index = pd.to_datetime(nohyeong_hierarchy_df.index)
nohyeong_hierarchy_df = nohyeong_hierarchy_df.resample("MS").sum()
# 대륜동
daeryun_df_bottom_level = daeryun_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
daeryun_df_total = daeryun_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
daeryun_hierarchy_df = daeryun_df_bottom_level.join(daeryun_df_total)
daeryun_hierarchy_df.index = pd.to_datetime(daeryun_hierarchy_df.index)
daeryun_hierarchy_df = daeryun_hierarchy_df.resample("MS").sum()
# 대정읍
daejeong_df_bottom_level = daejeong_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
daejeong_df_total = daejeong_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
daejeong_hierarchy_df = daejeong_df_bottom_level.join(daejeong_df_total)
daejeong_hierarchy_df.index = pd.to_datetime(daejeong_hierarchy_df.index)
daejeong_hierarchy_df = daejeong_hierarchy_df.resample("MS").sum()
# 대천동
daecheon_df_bottom_level = daecheon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
daecheon_df_total = daecheon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
daecheon_hierarchy_df = daecheon_df_bottom_level.join(daecheon_df_total)
daecheon_hierarchy_df.index = pd.to_datetime(daecheon_hierarchy_df.index)
daecheon_hierarchy_df = daecheon_hierarchy_df.resample("MS").sum()
# 도두동
dodu_df_bottom_level = dodu_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
dodu_df_total = dodu_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
dodu_hierarchy_df = dodu_df_bottom_level.join(dodu_df_total)
dodu_hierarchy_df.index = pd.to_datetime(dodu_hierarchy_df.index)
dodu_hierarchy_df = dodu_hierarchy_df.resample("MS").sum()
# 동홍동
donghong_df_bottom_level = donghong_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
donghong_df_total = donghong_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
donghong_hierarchy_df = donghong_df_bottom_level.join(donghong_df_total)
donghong_hierarchy_df.index = pd.to_datetime(donghong_hierarchy_df.index)
donghong_hierarchy_df = donghong_hierarchy_df.resample("MS").sum()
# 봉개동
bonggae_df_bottom_level = bonggae_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
bonggae_df_total = bonggae_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
bonggae_hierarchy_df = bonggae_df_bottom_level.join(bonggae_df_total)
bonggae_hierarchy_df.index = pd.to_datetime(bonggae_hierarchy_df.index)
bonggae_hierarchy_df = bonggae_hierarchy_df.resample("MS").sum()
# 삼도1동
samdo1_df_bottom_level = samdo1_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
samdo1_df_total = samdo1_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
samdo1_hierarchy_df = samdo1_df_bottom_level.join(samdo1_df_total)
samdo1_hierarchy_df.index = pd.to_datetime(samdo1_hierarchy_df.index)
samdo1_hierarchy_df = samdo1_hierarchy_df.resample("MS").sum()
# 삼도2동
samdo2_df_bottom_level = samdo2_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
samdo2_df_total = samdo2_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
samdo2_hierarchy_df = samdo2_df_bottom_level.join(samdo2_df_total)
samdo2_hierarchy_df.index = pd.to_datetime(samdo2_hierarchy_df.index)
samdo2_hierarchy_df = samdo2_hierarchy_df.resample("MS").sum()
# 삼양동
samyang_df_bottom_level = samyang_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
samyang_df_total = samyang_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
samyang_hierarchy_df = samyang_df_bottom_level.join(samyang_df_total)
samyang_hierarchy_df.index = pd.to_datetime(samyang_hierarchy_df.index)
samyang_hierarchy_df = samyang_hierarchy_df.resample("MS").sum()
# 서홍동
seohong_df_bottom_level = seohong_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
seohong_df_total = seohong_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
seohong_hierarchy_df = seohong_df_bottom_level.join(seohong_df_total)
seohong_hierarchy_df.index = pd.to_datetime(seohong_hierarchy_df.index)
seohong_hierarchy_df = seohong_hierarchy_df.resample("MS").sum()
# 성산읍
seongsan_df_bottom_level = seongsan_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
seongsan_df_total = seongsan_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
seongsan_hierarchy_df = seongsan_df_bottom_level.join(seongsan_df_total)
seongsan_hierarchy_df.index = pd.to_datetime(seongsan_hierarchy_df.index)
seongsan_hierarchy_df = seongsan_hierarchy_df.resample("MS").sum()
# 송산동
songsan_df_bottom_level = songsan_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
songsan_df_total = songsan_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
songsan_hierarchy_df = songsan_df_bottom_level.join(songsan_df_total)
songsan_hierarchy_df.index = pd.to_datetime(songsan_hierarchy_df.index)
songsan_hierarchy_df = songsan_hierarchy_df.resample("MS").sum()
# 아라동
ara_df_bottom_level = ara_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
ara_df_total = ara_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
ara_hierarchy_df = ara_df_bottom_level.join(ara_df_total)
ara_hierarchy_df.index = pd.to_datetime(ara_hierarchy_df.index)
ara_hierarchy_df = ara_hierarchy_df.resample("MS").sum()
# 안덕면
andeok_df_bottom_level = andeok_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
andeok_df_total = andeok_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
andeok_hierarchy_df = andeok_df_bottom_level.join(andeok_df_total)
andeok_hierarchy_df.index = pd.to_datetime(andeok_hierarchy_df.index)
andeok_hierarchy_df = andeok_hierarchy_df.resample("MS").sum()
# 애월읍
aewol_df_bottom_level = aewol_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
aewol_df_total = aewol_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
aewol_hierarchy_df = aewol_df_bottom_level.join(aewol_df_total)
aewol_hierarchy_df.index = pd.to_datetime(aewol_hierarchy_df.index)
aewol_hierarchy_df = aewol_hierarchy_df.resample("MS").sum()
# 연동
yeon_df_bottom_level = yeon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
yeon_df_total = yeon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
yeon_hierarchy_df = yeon_df_bottom_level.join(yeon_df_total)
yeon_hierarchy_df.index = pd.to_datetime(yeon_hierarchy_df.index)
yeon_hierarchy_df = yeon_hierarchy_df.resample("MS").sum()
# 영천동
yeongcheon_df_bottom_level = yeongcheon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
yeongcheon_df_total = yeongcheon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
yeongcheon_hierarchy_df = yeongcheon_df_bottom_level.join(yeongcheon_df_total)
yeongcheon_hierarchy_df.index = pd.to_datetime(yeongcheon_hierarchy_df.index)
yeongcheon_hierarchy_df = yeongcheon_hierarchy_df.resample("MS").sum()
# 예래동
yerae_df_bottom_level = yerae_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
yerae_df_total = yerae_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
yerae_hierarchy_df = yerae_df_bottom_level.join(yerae_df_total)
yerae_hierarchy_df.index = pd.to_datetime(yerae_hierarchy_df.index)
yerae_hierarchy_df = yerae_hierarchy_df.resample("MS").sum()
# 오라동
ora_df_bottom_level = ora_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
ora_df_total = ora_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
ora_hierarchy_df = ora_df_bottom_level.join(ora_df_total)
ora_hierarchy_df.index = pd.to_datetime(ora_hierarchy_df.index)
ora_hierarchy_df = ora_hierarchy_df.resample("MS").sum()
# 외도동
oedo_df_bottom_level = oedo_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
oedo_df_total = oedo_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
oedo_hierarchy_df = oedo_df_bottom_level.join(oedo_df_total)
oedo_hierarchy_df.index = pd.to_datetime(oedo_hierarchy_df.index)
oedo_hierarchy_df = oedo_hierarchy_df.resample("MS").sum()
# 용담1동
yongdam1_df_bottom_level = yongdam1_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
yongdam1_df_total = yongdam1_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
yongdam1_hierarchy_df = yongdam1_df_bottom_level.join(yongdam1_df_total)
yongdam1_hierarchy_df.index = pd.to_datetime(yongdam1_hierarchy_df.index)
yongdam1_hierarchy_df = yongdam1_hierarchy_df.resample("MS").sum()
# 용담2동
yongdam2_df_bottom_level = yongdam2_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
yongdam2_df_total = yongdam2_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
yongdam2_hierarchy_df = yongdam2_df_bottom_level.join(yongdam2_df_total)
yongdam2_hierarchy_df.index = pd.to_datetime(yongdam2_hierarchy_df.index)
yongdam2_hierarchy_df = yongdam2_hierarchy_df.resample("MS").sum()
# 우도면
udo_df_bottom_level = udo_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
udo_df_total = udo_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
udo_hierarchy_df = udo_df_bottom_level.join(udo_df_total)
udo_hierarchy_df.index = pd.to_datetime(udo_hierarchy_df.index)
udo_hierarchy_df = udo_hierarchy_df.resample("MS").sum()
# 이도1동
ido1_df_bottom_level = ido1_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
ido1_df_total = ido1_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
ido1_hierarchy_df = ido1_df_bottom_level.join(ido1_df_total)
ido1_hierarchy_df.index = pd.to_datetime(ido1_hierarchy_df.index)
ido1_hierarchy_df = ido1_hierarchy_df.resample("MS").sum()
# 이도2동
ido2_df_bottom_level = ido2_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
ido2_df_total = ido2_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
ido2_hierarchy_df = ido2_df_bottom_level.join(ido2_df_total)
ido2_hierarchy_df.index = pd.to_datetime(ido2_hierarchy_df.index)
ido2_hierarchy_df = ido2_hierarchy_df.resample("MS").sum()
# 이호동
iho_df_bottom_level = iho_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
iho_df_total = iho_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
iho_hierarchy_df = iho_df_bottom_level.join(iho_df_total)
iho_hierarchy_df.index = pd.to_datetime(iho_hierarchy_df.index)
iho_hierarchy_df = iho_hierarchy_df.resample("MS").sum()
# 일도1동
Ildo1_df_bottom_level = Ildo1_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
Ildo1_df_total = Ildo1_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
Ildo1_hierarchy_df = Ildo1_df_bottom_level.join(Ildo1_df_total)
Ildo1_hierarchy_df.index = pd.to_datetime(Ildo1_hierarchy_df.index)
Ildo1_hierarchy_df = Ildo1_hierarchy_df.resample("MS").sum()
# 일도2동
Ildo2_df_bottom_level = Ildo2_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
Ildo2_df_total = Ildo2_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
Ildo2_hierarchy_df = Ildo2_df_bottom_level.join(Ildo2_df_total)
Ildo2_hierarchy_df.index = pd.to_datetime(Ildo2_hierarchy_df.index)
Ildo2_hierarchy_df = Ildo2_hierarchy_df.resample("MS").sum()
# 정방동
jeongbang_df_bottom_level = jeongbang_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
jeongbang_df_total = jeongbang_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
jeongbang_hierarchy_df = jeongbang_df_bottom_level.join(jeongbang_df_total)
jeongbang_hierarchy_df.index = pd.to_datetime(jeongbang_hierarchy_df.index)
jeongbang_hierarchy_df = jeongbang_hierarchy_df.resample("MS").sum()
# 조천읍
jocheon_df_bottom_level = jocheon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
jocheon_df_total = jocheon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
jocheon_hierarchy_df = jocheon_df_bottom_level.join(jocheon_df_total)
jocheon_hierarchy_df.index = pd.to_datetime(jocheon_hierarchy_df.index)
jocheon_hierarchy_df = jocheon_hierarchy_df.resample("MS").sum()
# 중문동
jungmun_df_bottom_level = jungmun_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
jungmun_df_total = jungmun_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
jungmun_hierarchy_df = jungmun_df_bottom_level.join(jungmun_df_total)
jungmun_hierarchy_df.index = pd.to_datetime(jungmun_hierarchy_df.index)
jungmun_hierarchy_df = jungmun_hierarchy_df.resample("MS").sum()
# 중앙동
jungang_df_bottom_level = jungang_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
jungang_df_total = jungang_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
jungang_hierarchy_df = jungang_df_bottom_level.join(jungang_df_total)
jungang_hierarchy_df.index = pd.to_datetime(jungang_hierarchy_df.index)
jungang_hierarchy_df = jungang_hierarchy_df.resample("MS").sum()
# 천지동
cheonji_df_bottom_level = cheonji_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
cheonji_df_total = cheonji_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
cheonji_hierarchy_df = cheonji_df_bottom_level.join(cheonji_df_total)
cheonji_hierarchy_df.index = pd.to_datetime(cheonji_hierarchy_df.index)
cheonji_hierarchy_df = cheonji_hierarchy_df.resample("MS").sum()
# 추자면
chuja_df_bottom_level = chuja_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
chuja_df_total = chuja_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
chuja_hierarchy_df = chuja_df_bottom_level.join(chuja_df_total)
chuja_hierarchy_df.index = pd.to_datetime(chuja_hierarchy_df.index)
chuja_hierarchy_df = chuja_hierarchy_df.resample("MS").sum()
# 표선면
pyoseon_df_bottom_level = pyoseon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
pyoseon_df_total = pyoseon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
pyoseon_hierarchy_df = pyoseon_df_bottom_level.join(pyoseon_df_total)
pyoseon_hierarchy_df.index = pd.to_datetime(pyoseon_hierarchy_df.index)
pyoseon_hierarchy_df = pyoseon_hierarchy_df.resample("MS").sum()
# 한경면
hangyeong_df_bottom_level = hangyeong_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
hangyeong_df_total = hangyeong_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
hangyeong_hierarchy_df = hangyeong_df_bottom_level.join(hangyeong_df_total)
hangyeong_hierarchy_df.index = pd.to_datetime(hangyeong_hierarchy_df.index)
hangyeong_hierarchy_df = hangyeong_hierarchy_df.resample("MS").sum()
# 한림읍
hallim_df_bottom_level = hallim_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
hallim_df_total = hallim_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
hallim_hierarchy_df = hallim_df_bottom_level.join(hallim_df_total)
hallim_hierarchy_df.index = pd.to_datetime(hallim_hierarchy_df.index)
hallim_hierarchy_df = hallim_hierarchy_df.resample("MS").sum()
# 화북동
hwabuk_df_bottom_level = hwabuk_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
hwabuk_df_total = hwabuk_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
hwabuk_hierarchy_df = hwabuk_df_bottom_level.join(hwabuk_df_total)
hwabuk_hierarchy_df.index = pd.to_datetime(hwabuk_hierarchy_df.index)
hwabuk_hierarchy_df = hwabuk_hierarchy_df.resample("MS").sum()
# 효돈동
hyodon_df_bottom_level = hyodon_df.pivot(index = "m",columns = "nationality_group",values = "sum_resd_pop_cnt")
hyodon_df_total = hyodon_df.groupby("m")["sum_resd_pop_cnt"].sum().to_frame().rename(columns = {"sum_resd_pop_cnt" : "resd_total"})
hyodon_hierarchy_df = hyodon_df_bottom_level.join(hyodon_df_total)
hyodon_hierarchy_df.index = pd.to_datetime(hyodon_hierarchy_df.index)
hyodon_hierarchy_df = hyodon_hierarchy_df.resample("MS").sum()

3. 계층구조 만들기

### month_short_term_frgn 단기체류

In [ ]:
short_df = pd.read_csv('month_short_term_frgn.csv')
short_df

### 1. grouping

In [ ]:
group1= ['VNM']
group2 = ['JPN','SGP','IDN','MYS']
group3 = ['USA','CHN','ETC']
group4 = ['HKG','THA']

In [ ]:
for i in short_df.index:
    if short_df.loc[i,'nationality'] in group1:
        short_df.loc[i,'nationality_group'] = 'group1'
    elif short_df.loc[i,'nationality'] in group2:
        short_df.loc[i,'nationality_group'] = 'group2'
    elif short_df.loc[i,'nationality'] in group3:
        short_df.loc[i,'nationality_group'] = 'group3'
    elif short_df.loc[i,'nationality'] in group4:
        short_df.loc[i,'nationality_group'] = 'group4'


In [ ]:
short_df['nationality_group'].value_counts()

### month_resident_pop (거주 인구 데이터)

In [ ]:
# sex 그루핑 이미 나눠져 있음 !
resi_df = pd.read_csv('month_resident_pop.csv')
resi_df

### month_korean(내국인 유동인구 데이터)

In [ ]:
korean = pd.read_csv('month_korean.csv')
korean

In [ ]:
korean.info()

### 1. age grouping

In [ ]:
group1= [0,10]
group2 = [20,30]
group3 = [40,50]
group4 = [60]
group5 = [70,80]

In [ ]:
for i in korean.index:
    if korean.loc[i,'age'] in group1:
        korean.loc[i,'age_group'] = 'group1'
    elif korean.loc[i,'age'] in group2:
        korean.loc[i,'age_group'] = 'group2'
    elif korean.loc[i,'age'] in group3:
        korean.loc[i,'age_group'] = 'group3'
    elif korean.loc[i,'age'] in group4:
        korean.loc[i,'age_group'] = 'group4'
    elif korean.loc[i,'age'] in group5:
        korean.loc[i,'age_group'] = 'group5'

In [ ]:
korean['age_group'].unique()

In [ ]:
korean['age_group'].value_counts()

### month_card_spending (음식 관련 카드소비 데이터)

In [ ]:
card_df = pd.read_csv('month_card_spending.csv',encoding='cp949')
card_df

### 1. mct_cat_nm grouping

In [ ]:
card_df['mct_cat_nm'].unique()

In [ ]:
group1 = ['마트/슈퍼마켓','한식']
group2 = ['식품']
group3 = ['농축수산물']
group4 = ['아시아음식','간식','배달','패스트푸드','양식','부페','주점및주류판매']

In [ ]:
for i in card_df.index:
    if card_df.loc[i,'mct_cat_nm'] in group1:
        card_df.loc[i,'mct_cat_nm_group'] = 'group1'
    elif card_df.loc[i,'mct_cat_nm'] in group2:
        card_df.loc[i,'mct_cat_nm_group'] = 'group2'
    elif card_df.loc[i,'mct_cat_nm'] in group3:
        card_df.loc[i,'mct_cat_nm_group'] = 'group3'
    elif card_df.loc[i,'mct_cat_nm'] in group4:
        card_df.loc[i,'mct_cat_nm_group'] = 'group4'

In [ ]:
card_df['mct_cat_nm_group'].unique()

In [ ]:
card_df['mct_cat_nm_group'].value_counts()